In [ ]:
pip install openai

In [ ]:
pip install telebot

In [ ]:
import telebot
from telebot import types
from openai import OpenAI
import json
import asyncio
import google.generativeai as genai
import os
import requests
import io

telegram_token = ""
bot = telebot.TeleBot(telegram_token, parse_mode=None, threaded=False)

client = OpenAI(
    api_key=""
)

history = []
theme = ''
players = ''
conflict = ''
level = 0
duration = 0
@bot.message_handler(commands=['start'])
def start(message: types.Message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("Новая игра")
    markup.add(btn1)
    bot.send_message(message.chat.id, text="Привет! Я могу быть Dungeons&Dragons Game Master и помогать в ведении игры: придумывать сюжет, добавлять визуализацию и озвучку. Нажми 'Новая Игра', чтобы начать.", reply_markup=markup)

@bot.message_handler(content_types=['text'])
def func(message):
    global theme
    global players
    global conflict
    global duration
    global level
    if message.text == "Новая игра":
        history.clear()
        chat_id = message.chat.id
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        btn1 = types.KeyboardButton("Фэнтези")
        btn2 = types.KeyboardButton("Хоррор")
        btn3 = types.KeyboardButton("Мифология")
        btn4 = types.KeyboardButton("Приключения")
        btn5 = types.KeyboardButton("Мистика")
        btn6 = types.KeyboardButton("Новая игра")
        markup.add(btn1, btn2, btn3, btn4, btn5, btn6)
        bot.send_message(chat_id, "Выберите жанр", reply_markup=markup)
        bot.register_next_step_handler(message, select_genre)
def select_genre(message):
    if message.text=="Новая игра":
      start(message)
    else:
      global theme
      theme = message.text
      bot.reply_to(message, f'Вы выбрали жанр: {theme}')
      markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
      btn = types.KeyboardButton("Новая игра")
      markup.add(btn)
      bot.send_message(message.chat.id, "Введите количество игроков", reply_markup=markup)
      bot.register_next_step_handler(message, select_players)

def select_players(message):
    if message.text=="Новая игра":
      start(message)
    else:
      global players
      players = message.text
      bot.reply_to(message, f'Количество игроков: {players}')
      markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
      btn1 = types.KeyboardButton("1-2")
      btn2 = types.KeyboardButton("3-5")
      btn3 = types.KeyboardButton("6-10")
      btn4 = types.KeyboardButton("11-15")
      btn5 = types.KeyboardButton("16-20")
      btn6 = types.KeyboardButton("Новая игра")
      markup.add(btn1, btn2, btn3, btn4, btn5, btn6)
      bot.send_message(message.chat.id, "Выберите уровень", reply_markup=markup)
      bot.register_next_step_handler(message, select_level)

def select_level(message):
    if message.text=="Новая игра":
      start(message)
    else:
      global level
      level = message.text
      bot.reply_to(message, f'Уровень: {level}')
      markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
      btn1 = types.KeyboardButton("Добро против зла")
      btn2 = types.KeyboardButton("Личностные конфликты")
      btn3 = types.KeyboardButton("Цивилизация и природа")
      btn4 = types.KeyboardButton("Выживание")
      btn5 = types.KeyboardButton("Этнические конфликты")
      btn6 = types.KeyboardButton("Новая игра")
      markup.add(btn1, btn2, btn3, btn4, btn5, btn6)
      bot.send_message(message.chat.id, "Выберите основной конфликт", reply_markup=markup)
      bot.register_next_step_handler(message, select_conflict)

def select_conflict(message):
    if message.text=="Новая игра":
      start(message)
    else:
      global conflict
      conflict = message.text
      bot.reply_to(message, f'Основной конфликт: {conflict}')
      markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
      btn1 = types.KeyboardButton("One-shot (3-5 часов)")
      btn2 = types.KeyboardButton("До 10 сессий")
      btn3 = types.KeyboardButton("Больше 10 сессий")
      btn4 = types.KeyboardButton("Новая игра")
      markup.add(btn1, btn2, btn3, btn4)
      bot.send_message(message.chat.id, "Выберите длительность кампании", reply_markup=markup)
      bot.register_next_step_handler(message, select_duration)
def select_duration(message):
    if message.text=="Новая игра":
      start(message)
    else:
      global duration
      duration= message.text
      bot.reply_to(message, f'Длительность: {duration}')

      bot.send_chat_action(message.chat.id, "typing")
      history.append({"role": "user",
                      "content": f"Ты - мастер игры Поздемелья и Драконы. Создай уникальный мир для нашего приключения в жанре {theme} для {players} игроков уровнем {level} длительностью {duration} с основным конфликтом - {conflict}. Напиши название мира и его краткое описание."})
      response = client.chat.completions.create(
          model="gpt-4o-mini",
          messages=history,
      )
      history.append({"role": "assistant", "content": response.choices[0].message.content})
      markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
      btn1 = types.KeyboardButton("Создать персонажей")
      btn2 = types.KeyboardButton("Новая игра")
      markup.add(btn1, btn2)
      text = response.choices[0].message.content
      bot.send_message(message.chat.id, text)
      bot.register_next_step_handler(message, characters)
      bot.send_chat_action(message.chat.id, "upload_photo")
      response = client.images.generate(
        model="dall-e-3",
        prompt=f'Картинка без текста. {text}',
        size="1024x1024",
        quality="standard",
        n=1,
      )
      bot.send_chat_action(message.chat.id, "upload_photo")
      bot.send_photo(message.chat.id, response.data[0].url)

      bot.send_chat_action(message.chat.id, "upload_audio")

      response = client.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=text,
      )

      speech_file_path = 'output.mp3'
      response.stream_to_file(speech_file_path)
      bot.send_chat_action(message.chat.id, "upload_audio")
      with open(speech_file_path, 'rb') as audio_file:
          bot.send_audio(message.chat.id, audio_file, title=f"Сцена", performer="D&D", reply_markup=markup)

      os.remove(speech_file_path)


def characters(message):
    if message.text=="Новая игра":
      func(message)
    else:
      global players
      bot.send_chat_action(message.chat.id, "typing")
      history.append({"role": "user",
                      "content": f"Создай для этого мира {players} персонажей."
                                f" Для каждого персонажа кратко напиши его имя, рост, вес, возраст, внешний вид, мировоззрение, предыстория, экипировка, расу, класс, значение и модификатор характеристик, навыки, "
                                f" кость хитов, бонус владения."})
      response = client.chat.completions.create(
          model="gpt-4o-mini",
          messages=history,
      )
      history.append({"role": "assistant", "content": response.choices[0].message.content})
      markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
      btn1 = types.KeyboardButton("Начать игру")
      btn2 = types.KeyboardButton("Новая игра")
      markup.add(btn1, btn2)
      text = response.choices[0].message.content
      bot.send_message(message.chat.id, text)
      bot.send_chat_action(message.chat.id, "upload_photo")
      response = client.images.generate(
        model="dall-e-3",
        prompt=text,
        size="1024x1024",
        quality="standard",
        n=1,
      )
      bot.send_photo(message.chat.id, response.data[0].url)

      bot.send_chat_action(message.chat.id, "upload_audio")

      response = client.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=text,
      )

      speech_file_path = 'output.mp3'
      response.stream_to_file(speech_file_path)
      bot.send_chat_action(message.chat.id, "upload_audio")
      with open(speech_file_path, 'rb') as audio_file:
          bot.send_audio(message.chat.id, audio_file, title=f"Сцена", performer="D&D", reply_markup=markup)
      os.remove(speech_file_path)

    bot.register_next_step_handler(message, game)
def game(message):
    if message.text=="Новая игра":
      start(message)
    else:
      global duration
      history.append({"role": "user",
                      "content": f"Ты - DM. Придумай сюжет длительностью {duration} учитывая количество и уровень персонажей. Опиши первую сцену. Для монстра опиши размер, вид, мировоззрение, класс доспеха, хиты, скорость, значение характеристик, спасброски, опасность, навыки, уязвимости, иммунитеты, чувства, языки, опыт, особенности, снаряжение монстра. Предложи возможные действия персонажей. Ты можешь просить игроков бросить игральную кость."})
      response = client.chat.completions.create(
          model="gpt-4o-mini",
          messages=history,
      )
      bot.send_chat_action(message.chat.id, "typing")
      text = response.choices[0].message.content
      history.append({"role": "assistant", "content": f"{text} Какие ваши действия?"})
      bot.send_message(message.chat.id, text)
      bot.send_chat_action(message.chat.id, "upload_photo")
      response = client.images.generate(
        model="dall-e-3",
        prompt=text,
        size="1024x1024",
        quality="standard",
        n=1,
      )
      bot.send_photo(message.chat.id, response.data[0].url)

      bot.send_chat_action(message.chat.id, "upload_audio")

      response = client.audio.speech.create(
        model="tts-1",
        voice="onyxy",
        input=text,
      )

      speech_file_path = 'output.mp3'
      response.stream_to_file(speech_file_path)
      bot.send_chat_action(message.chat.id, "upload_audio")
      with open(speech_file_path, 'rb') as audio_file:
          bot.send_audio(message.chat.id, audio_file, title=f"Сцена", performer="D&D", reply_markup=markup)
      os.remove(speech_file_path)

      bot.register_next_step_handler(message, action)
def action(message):
    if message.text == "Приключение закончилось":
      markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
      btn1 = types.KeyboardButton("Новая игра")
      markup.add(btn1)
      bot.send_message(message.chat.id, text="На этом все!")
      bot.register_next_step_handler(message, func)
    bot.send_chat_action(message.chat.id, "typing")
    history.append({"role": "user",
                    "content": f"Ты - мастер игры. Веди сюжет дальше по действиям игроков: {message}. Если сюжет уже подошел к концу, напиши только 'Приключение закончилось'. Пиши текст до 4000 символов."})
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=history,
    )
    bot.send_chat_action(message.chat.id, "typing")
    text = response.choices[0].message.content
    history.append({"role": "assistant", "content": f"{text} Какие ваши действия?"})
    bot.send_message(message.chat.id, text)
    bot.send_chat_action(message.chat.id, "upload_photo")
    response = client.images.generate(
      model="dall-e-3",
      prompt=text,
      size="1024x1024",
      quality="standard",
      n=1,
    )
    bot.send_photo(message.chat.id, response.data[0].url, reply_markup=markup)

    bot.send_chat_action(message.chat.id, "upload_audio")

    response = client.audio.speech.create(
      model="tts-1",
      voice="onyx",
      input=text,
    )

    speech_file_path = 'output.mp3'
    response.stream_to_file(speech_file_path)
    bot.send_chat_action(message.chat.id, "upload_audio")
    with open(speech_file_path, 'rb') as audio_file:
        bot.send_audio(message.chat.id, audio_file, title=f"Сцена", performer="D&D", reply_markup=markup)
    os.remove(speech_file_path)
    bot.register_next_step_handler(message, action)


if __name__ == '__main__':
    bot.infinity_polling(timeout=10, long_polling_timeout = 5)